In [15]:
import pandas as pd
from sklearn.metrics import f1_score, classification_report, accuracy_score

In [16]:
data_root = "data/llama_whole_new_direct/"
base_path = "_llama.csv"
k0 = pd.read_csv(data_root + "IE" + base_path, delimiter='\t')
k0 = k0[['pred','label0']]
k0 = k0.rename(columns={'pred':'pred','label0':'gold'})

k1 = pd.read_csv(data_root + "NS" + base_path, delimiter='\t')
k1 = k1[['pred','label1']]
k1 = k1.rename(columns={'pred':'pred','label1':'gold'})

k2 = pd.read_csv(data_root + "TF" + base_path, delimiter='\t')
k2 = k2[['pred','label2']]
k2 = k2.rename(columns={'pred':'pred','label2':'gold'})

k3 = pd.read_csv(data_root + "JP" + base_path, delimiter='\t')
k3 = k3[['pred','label3']]
k3 = k3.rename(columns={'pred':'pred','label3':'gold'})

tags = []

In [17]:
def modify1(new_a):
    new_a = new_a.replace('.*[iI]nt.*',0, regex=True)
    new_a = new_a.replace('.*IN.*',0, regex=True)
    new_a = new_a.replace('.*ext.*',1, regex=True)
    new_a = new_a.replace('.*am.*',2, regex=True)
    new_a = new_a.replace('.*both.*',2, regex=True)
    new_a = new_a.replace('.*neither.*',2, regex=True)
    new_a = new_a.replace('.*[^01].*',2, regex=True)
    new_a = new_a.fillna(2)
    return new_a
tags.append(modify1(k0))
def modify2(new_a):
    new_a = new_a.replace('.*ens.*',1, regex=True)
    new_a = new_a.replace('.*eel.*',1, regex=True)
    new_a = new_a.replace('.*tui.*',0, regex=True)
    new_a = new_a.replace('.*both.*',2, regex=True)
    new_a = new_a.replace('.*[Nn].*',0, regex=True)
    new_a = new_a.replace('.*m.*',2, regex=True)
    new_a = new_a.fillna(2)
    return new_a
tags.append(modify2(k1))
def modify3(new_a):
    new_a = new_a.replace('.*[Hh]ybrid.*',2, regex=True)
    new_a = new_a.replace('.*eel.*',0, regex=True)
    new_a = new_a.replace('.*ink.*',1, regex=True)
    new_a = new_a.replace('.*T.*',1, regex=True)
    new_a = new_a.replace('.*[Nn].*',2, regex=True)
    new_a = new_a.replace('.*Ambi.*',2, regex=True)
    new_a = new_a.replace('.*ambi.*',2, regex=True)
    new_a = new_a.replace('.*both.*',2, regex=True)
    new_a = new_a.fillna(2)
    return new_a
tags.append(modify3(k2))
def modify4(new_a):
    new_a = new_a.replace('.*[Pp].*',0, regex=True)
    new_a = new_a.replace('.*[Jj].*',1, regex=True)
    new_a = new_a.replace('.*neutral.*',2, regex=True)
    new_a = new_a.replace('.*ambi.*',2, regex=True)
    new_a = new_a.replace('.*mix.*',2, regex=True)
    new_a = new_a.replace('.*both.*',2, regex=True)
    new_a = new_a.replace('.*N/A.*',2, regex=True)
    new_a = new_a.replace('.*[^01].*',2, regex=True)
    # new_a = new_a.replace('.*T.*',1, regex=True)
    new_a = new_a.fillna(2)
    return new_a
tags.append(modify4(k3))
# k0 = modify(k0)

In [18]:
from collections import Counter
print(Counter(tags[0]['pred']))
print(Counter(tags[1]['pred']))
print(Counter(tags[2]['pred']))
print(Counter(tags[3]['pred']))

Counter({0: 175, 1: 24, 2: 1})
Counter({0: 165, 1: 35})
Counter({1: 114, 0: 86})
Counter({1.0: 110, 0.0: 88, 2.0: 2})


In [19]:
macro_f1 = []
acc = []
for i in tags:
    print(classification_report(i['gold'],i['pred'],digits=4, labels=[0, 1]))
    macro_f1.append(f1_score(i['gold'],i['pred'],average='macro', labels=[0,1]))
    acc.append(accuracy_score(i['gold'],i['pred']))

              precision    recall  f1-score   support

           0     0.7829    0.9013    0.8379       152
           1     0.4167    0.2083    0.2778        48

   micro avg     0.7387    0.7350    0.7368       200
   macro avg     0.5998    0.5548    0.5578       200
weighted avg     0.6950    0.7350    0.7035       200

              precision    recall  f1-score   support

           0     0.8788    0.8382    0.8580       173
           1     0.2000    0.2593    0.2258        27

    accuracy                         0.7600       200
   macro avg     0.5394    0.5487    0.5419       200
weighted avg     0.7872    0.7600    0.7726       200

              precision    recall  f1-score   support

           0     0.7907    0.5913    0.6766       115
           1     0.5877    0.7882    0.6734        85

    accuracy                         0.6750       200
   macro avg     0.6892    0.6898    0.6750       200
weighted avg     0.7044    0.6750    0.6752       200

              preci

In [20]:
print(acc, sum(acc)/len(acc))
print(macro_f1, sum(macro_f1)/len(macro_f1))
output = []
for i, j in zip(macro_f1,acc):
    output.append(round(j*100,2))
    output.append(round(i*100,2))
output.append(round(sum(acc)/len(acc)*100,2))
output.append(round(sum(macro_f1)/len(macro_f1)*100,2))
output = [str(i) for i in output]
print(output)

[0.735, 0.76, 0.675, 0.52] 0.6725
[0.5578491335372069, 0.5418973086466883, 0.67499187479687, 0.520871327254306] 0.5739024110587678
['73.5', '55.78', '76.0', '54.19', '67.5', '67.5', '52.0', '52.09', '67.25', '57.39']


In [21]:
print(' & '.join(output))

73.5 & 55.78 & 76.0 & 54.19 & 67.5 & 67.5 & 52.0 & 52.09 & 67.25 & 57.39
